In [1]:
current_File = "AutoencoderModels.ipynb"

In [3]:
import time
start_time = time.time() 

In [4]:
# pip install torch 
#  conda install pytorch torchvision cpuonly -c pytorch 
import torch
import torch.nn as nn
import torchvision
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F

## Autoencoder Models 

### Basic 

In [5]:
class scRNAseqAutoencoder_Basic(nn.Module):
    def __init__(self, outer_layer=1536, inner_layer=64):
        super(scRNAseqAutoencoder_Basic, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(outer_layer, 512),
            nn.ReLU(True),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Linear(128, inner_layer))  # Compressed representation
        self.decoder = nn.Sequential(
            nn.Linear(inner_layer, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, outer_layer),
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


### Denoising  

In [6]:
class scRNAseqAutoencoder_Denoising(nn.Module):
    def __init__(self, outer_layer = 1536, inner_layer = 64):
        super(scRNAseqAutoencoder_Denoising, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(outer_layer, 512),
            nn.ReLU(True),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Linear(128, inner_layer))  # Compressed representation
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(inner_layer, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, outer_layer),
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


### Dropout  

In [7]:
class scRNAseqAutoencoder_Dropout(nn.Module):
    def __init__(self, outer_layer = 1536, inner_layer = 64, dropout_rate = 0.2):
        super(scRNAseqAutoencoder_Dropout, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(outer_layer, 512),
            nn.ReLU(True),
            nn.Dropout(p=dropout_rate),  # Dropout layer with a probability of 20%
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(p=dropout_rate),  # Dropout layer with a probability of 20%
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Linear(128, inner_layer))  # Compressed representation
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(inner_layer, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.Dropout(p=dropout_rate),  # Dropout layer with a probability of 20%
            nn.ReLU(True),
            nn.Linear(512, outer_layer),
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
class scRNAseqAutoencoder_Combined_Dropout(nn.Module):
    def __init__(self, input_outer_layer=3072, output_outer_layer=100, inner_layer=64, dropout_rate=0.2):  # Adjusted input_outer_layer to 3072
        super(scRNAseqAutoencoder_Combined_Dropout, self).__init__()
        print("Adjusted model for concatenated embeddings")
        input_outer_layer=3072
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_outer_layer, 512),  # Adjusted to match the concatenated input size
            nn.ReLU(True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Linear(128, inner_layer)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(inner_layer, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.Dropout(p=dropout_rate),
            nn.ReLU(True),
            nn.Linear(512, input_outer_layer),  # This will now correctly aim to reconstruct the 3072-sized input
            nn.Tanh()
        )

        # Dimension Reductor
        self.reductor = nn.Sequential(
            nn.Linear(input_outer_layer, 512),
            nn.ReLU(True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Linear(128, 100)
        )

        

    def forward(self, x):
        #print("Control Point 2.0")
        x = self.encoder(x)
        #print("Control Point 2.1")
        x = self.decoder(x)
        y = x.clone() 
        #print("Control Point 2.2")
        x = self.reductor(x) 
        #print("Control Point 2.3")
        return y,x


## Manager   

In [8]:
class AutoencoderManager:
    def __init__(self, 
                model_type='basic', 
                lr=1e-3, 
                batch_size=128, 
                num_epochs=5, 
                noise_factor=0.3,
                dropout_rate = 0.2, 
                outer_layer = 1536,
                inner_layer = 64,
                input_outer_layer = 1536, 
                output_outer_layer = 100 
                ):
        
        # Combined Dropout 
        self.input_outer_layer  = input_outer_layer 
        self.output_outer_layer = output_outer_layer 
        
        self.output_layer = outer_layer
        self.inner_layer = inner_layer 
        self.model_type = model_type
        self.noise_factor = noise_factor
        self.dropout_rate = dropout_rate 
        self.model = self._get_model()
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.train_loader = None  # Initialized in load_data

    def _get_model(self):
        if self.model_type == 'basic':
            return scRNAseqAutoencoder_Basic(outer_layer = self.output_layer, 
                                             inner_layer = self.inner_layer)
        elif self.model_type == 'denoising':
            return scRNAseqAutoencoder_Denoising(outer_layer = self.output_layer, 
                                                 inner_layer = self.inner_layer)
        elif self.model_type == 'dropout' or self.model_type == 'combined':
            return scRNAseqAutoencoder_Dropout(outer_layer = self.output_layer, 
                                               inner_layer = self.inner_layer, 
                                               dropout_rate = self.dropout_rate)
        elif self.model_type == 'combined_dropout':
            return scRNAseqAutoencoder_Combined_Dropout(input_outer_layer  = self.input_outer_layer,
                                                        output_outer_layer = self.output_outer_layer, 
                                                        inner_layer  = self.inner_layer, 
                                                        dropout_rate = self.dropout_rate) 

        else:
            raise ValueError("Invalid model type specified")

    def load_data(self, data_matrix, label_matrix=None):
        # Convert matrices to tensors if they're not already
        data_tensor = torch.tensor(data_matrix).float()
        if label_matrix is not None:
            label_tensor = torch.tensor(label_matrix).float()
        else:
            # If no labels are provided, use the data itself as labels (unsupervised learning)
            label_tensor = data_tensor
        
        # Create a dataset and loader for the tensors
        dataset = TensorDataset(data_tensor, label_tensor)
        self.train_loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

    def train(self):
        if self.train_loader is None:
            raise ValueError("Data not loaded. Call load_data before training.")
        for epoch in range(self.num_epochs):
            for data in self.train_loader:
                inputs, targets = data
                if self.model_type == 'denoising' or self.model_type == 'combined':
                    # Add noise only for the denoising model
                    inputs = add_noise(inputs, self.noise_factor)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            print(f'Epoch [{epoch+1}/{self.num_epochs}], Loss: {loss.item():.4f}')

    
    def get_encoded_representations(self, data):
        self.model.eval()
        data_tensor = torch.tensor(data).float()
        # If using a GPU
        data_tensor = data_tensor.to(next(self.model.parameters()).device)  # Match model device
        with torch.no_grad():
            encoded_representations = self.model.encoder(data_tensor)
        return encoded_representations.cpu().numpy()  

    def get_reconstructed_data(self, data): # CPU 
        self.model.eval()
        data_tensor = torch.tensor(data).float()
        # If using a GPU
        data_tensor = data_tensor.to(next(self.model.parameters()).device)  # Match model device
        with torch.no_grad():
            reconstructed_data = self.model(data_tensor)
        return reconstructed_data.cpu().numpy() 


    def train_combined_dropout(self):
        print("Training combined dropout model 1.0")
        if self.train_loader is None:
            raise ValueError("Data not loaded. Call load_data before training.")
        print("Training combined dropout model 1.1")
        self.model.train()  # Ensure the model is in training mode
        print("Training combined dropout model 1.2")
        for epoch in range(self.num_epochs):
            total_loss = 0
            
            for data in self.train_loader:
                inputs, _ = data  # Assuming autoencoder style where input = target
                self.optimizer.zero_grad()

                # Forward pass
                reconstructed, reduced = self.model(inputs)
                
                # Compute loss only with reconstructed part, as reduced is for dimensionality reduction
                loss = self.criterion(reconstructed, inputs)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()

            avg_loss = total_loss / len(self.train_loader)
            print(f'Epoch [{epoch+1}/{self.num_epochs}], Loss: {avg_loss:.4f}')

        print("Training combined dropout model 1.3") 
        
    def get_reconstructed_data_combined_dropout(self, data): 
        self.model.eval()
        reduced_outputs = []  # List to store outputs for each batch

        # Assuming 'data' is your input matrix of shape (1690, 3072) or similar
        data_tensor = torch.tensor(data).float().to(next(self.model.parameters()).device)

        with torch.no_grad():
            # Assuming your data is not already in DataLoader format
            data_loader = DataLoader(data_tensor, batch_size=self.batch_size, shuffle=False)
            
            for batch in data_loader:
                _, reduced = self.model(batch)  # Get the reduced representation
                reduced_outputs.append(reduced)

        # Concatenate all reduced representations
        reduced_matrix = torch.cat(reduced_outputs, dim=0)
        return reduced_matrix.cpu().numpy()



# Helper functions outside the class
def add_noise(images, noise_factor=0.3):
    noisy_images = images + noise_factor * torch.randn_like(images)
    noisy_images = torch.clamp(noisy_images, 0., 1.)
    return noisy_images

In [9]:
class static_autoencoder():

    @staticmethod 
    def basic(target_matrix, 
                # variables: learning rate
                model_type   = 'basic' ,
                variable_lr  = [1e-3,1e-4,1e-5] ,
                batch_size   = 128,
                num_epochs   = 5,
                noise_factor = 0.3,
                dropout_rate = 0.2 ,
                outer_layer  = 1536,
                inner_layer  = 64 ,
        ):  

        B1_a_manager = AutoencoderManager(  model_type   = 'basic', 
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor,
                                    dropout_rate = dropout_rate, 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer) 
                                    
        B1_a_manager.load_data( data_matrix = target_matrix, 
        )   
        B1_a_manager.train() 
        encoded_representations = B1_a_manager.get_encoded_representations(target_matrix)
        B1_a_reconstructed_data = B1_a_manager.get_reconstructed_data(target_matrix)


        B2_b_manager = AutoencoderManager(  model_type   = 'basic', 
                                    lr           = variable_lr[1], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor,
                                    dropout_rate = dropout_rate, 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)  

        B2_b_manager.load_data( data_matrix = target_matrix, 
        ) 
        B2_b_manager.train()
        B2_b_reconstructed_data = B2_b_manager.get_reconstructed_data(target_matrix) 


                            
        B2_c_manager = AutoencoderManager(  model_type   = 'basic', 
                                            lr           = variable_lr[2], 
                                            batch_size   = batch_size, 
                                            num_epochs   = num_epochs, 
                                            noise_factor = noise_factor,
                                            dropout_rate = dropout_rate, 
                                            outer_layer  = outer_layer, 
                                            inner_layer  = inner_layer,
                                    )  

        B2_c_manager.load_data( data_matrix = target_matrix,
        )
        B2_c_manager.train()
        B2_c_reconstructed_data = B2_c_manager.get_reconstructed_data(target_matrix) 
                
        B1_List = B1_a_reconstructed_data, B2_b_reconstructed_data, B2_c_reconstructed_data 
        return B1_List 

    @staticmethod
    def denoising(target_matrix , 
        # variables: learning rate, noise_factor
        model_type   = 'denoising' ,
        variable_lr  = [1e-3, 1e-4] ,
        batch_size   = 128,
        num_epochs   = 5,
        noise_factor = [0.3,0.4],
        dropout_rate = 0.0 ,
        outer_layer  = 1536,
        inner_layer  = 64 ,
        ): 

        B2_a_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[0],
                                    dropout_rate = dropout_rate, 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)   
        B2_a_manager.load_data( data_matrix = target_matrix,
        )
        B2_a_manager.train()
        B2_a_reconstructed_data = B2_a_manager.get_reconstructed_data(target_matrix) 



        B2_b_manager = AutoencoderManager(  model_type   = model_type,     
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[1],
                                    dropout_rate = dropout_rate, 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)  

        B2_b_manager.load_data( data_matrix = target_matrix,
        ) 
        B2_b_manager.train()
        B2_b_reconstructed_data = B2_b_manager.get_reconstructed_data(target_matrix) 

                                    

        B2_c_manager = AutoencoderManager(  model_type   = model_type,     
                                    lr           = variable_lr[1], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[0],
                                    dropout_rate = dropout_rate, 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)         

        B2_c_manager.load_data( data_matrix = target_matrix,
        )
        B2_c_manager.train()
        B2_c_reconstructed_data = B2_c_manager.get_reconstructed_data(target_matrix)
         

        B2_d_manager = AutoencoderManager(  model_type   = model_type,
                                            lr           = variable_lr[1], 
                                            batch_size   = batch_size, 
                                            num_epochs   = num_epochs, 
                                            noise_factor = noise_factor[1],
                                            dropout_rate = dropout_rate, 
                                            outer_layer  = outer_layer, 
                                            inner_layer  = inner_layer
                                    )        

        B2_d_manager.load_data( data_matrix = target_matrix,
        )
        B2_d_manager.train()
        B2_d_reconstructed_data = B2_d_manager.get_reconstructed_data(target_matrix) 

        B2_List = B2_a_reconstructed_data, B2_b_reconstructed_data, B2_c_reconstructed_data, B2_d_reconstructed_data     
        return B2_List 
        
    @staticmethod
    def dropout(target_matrix,
        # variables: learning rate, dropout_rate
                model_type   = 'dropout' ,
                variable_lr  = [1e-3, 1e-4] ,
                batch_size   = 128,
                num_epochs   = 5,
                noise_factor = 0 ,
                dropout_rate = [0.1,0.2],
                outer_layer  = 1536,
                inner_layer  = 64 ,
        ):

        B3_a_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor,
                                    dropout_rate = dropout_rate[0], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)      

        B3_a_manager.load_data( data_matrix = target_matrix, 
        ) 
        B3_a_manager.train() 
        B3_a_reconstructed_data = B3_a_manager.get_reconstructed_data(target_matrix) 



        B3_b_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor,
                                    dropout_rate = dropout_rate[1], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)     
                                    
        B3_b_manager.load_data( data_matrix = target_matrix ) 
        B3_b_manager.train()
        B3_b_reconstructed_data = B3_b_manager.get_reconstructed_data(target_matrix) 


        B3_c_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[1], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor,
                                    dropout_rate = dropout_rate[0], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer) 
        B3_c_manager.load_data( data_matrix = target_matrix ) 
        B3_c_manager.train() 
        B3_c_reconstructed_data = B3_c_manager.get_reconstructed_data(target_matrix) 

        B3_d_manager = AutoencoderManager(  model_type   = 'dropout',
                                    lr           = variable_lr[1], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor,
                                    dropout_rate = dropout_rate[1], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer) 
        B3_d_manager.load_data( data_matrix = target_matrix ) 
        B3_d_manager.train()
        B3_d_reconstructed_data = B3_d_manager.get_reconstructed_data(target_matrix) 

        B3_List = B3_a_reconstructed_data, B3_b_reconstructed_data, B3_c_reconstructed_data, B3_d_reconstructed_data 
        return B3_List 


    @staticmethod
    def combined(target_matrix,
                # variables: learning rate, noise_factor, dropout_rate 
                model_type   = 'combined'  ,
                variable_lr  = [1e-3, 1e-4], 
                batch_size   = 128,
                num_epochs   = 5,
                noise_factor = [0.3,0.4],
                dropout_rate = [0.1,0.2],
                outer_layer  = 1536,
                inner_layer  = 64 ,
                ):
        # Outputs B4_a, B4_b, B4_c, B4_d, B4_e, B4_f, B4_g, B4_h 

        B4_a_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[0],
                                    dropout_rate = dropout_rate[0], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer) 
        B4_a_manager.load_data( data_matrix = target_matrix ) 
        B4_a_manager.train() 
        B4_a_reconstructed_data = B4_a_manager.get_reconstructed_data(target_matrix) 



        B4_b_manager = AutoencoderManager(  model_type   = model_type,   
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[0],
                                    dropout_rate = dropout_rate[1], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)     
        B4_b_manager.load_data( data_matrix = target_matrix )   
        B4_b_manager.train()
        B4_b_reconstructed_data = B4_b_manager.get_reconstructed_data(target_matrix)



        B4_c_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[1],
                                    dropout_rate = dropout_rate[0], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)     
        B4_c_manager.load_data( data_matrix = target_matrix )   
        B4_c_manager.train()
        B4_c_reconstructed_data = B4_c_manager.get_reconstructed_data(target_matrix)



        B4_d_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[1],
                                    dropout_rate = dropout_rate[1], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer) 
        B4_d_manager.load_data( data_matrix = target_matrix )
        B4_d_manager.train()
        B4_d_reconstructed_data = B4_d_manager.get_reconstructed_data(target_matrix)

        B4_e_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[1], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[0],
                                    dropout_rate = dropout_rate[0], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)     
        B4_e_manager.load_data( data_matrix = target_matrix )
        B4_e_manager.train()
        B4_e_reconstructed_data = B4_e_manager.get_reconstructed_data(target_matrix)


        B4_f_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[1], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[0],
                                    dropout_rate = dropout_rate[1], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)     
        B4_f_manager.load_data( data_matrix = target_matrix )
        B4_f_manager.train()
        B4_f_reconstructed_data = B4_f_manager.get_reconstructed_data(target_matrix)



        B4_g_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[1], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[1],
                                    dropout_rate = dropout_rate[0], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer)
        B4_g_manager.load_data( data_matrix = target_matrix )
        B4_g_manager.train()
        B4_g_reconstructed_data = B4_g_manager.get_reconstructed_data(target_matrix) 



        B4_h_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[1], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[1],
                                    dropout_rate = dropout_rate[1], 
                                    outer_layer  = outer_layer, 
                                    inner_layer  = inner_layer) 
        B4_h_manager.load_data( data_matrix = target_matrix )
        B4_h_manager.train()
        B4_h_reconstructed_data = B4_h_manager.get_reconstructed_data(target_matrix)

        B4_List = B4_a_reconstructed_data, B4_b_reconstructed_data, B4_c_reconstructed_data, B4_d_reconstructed_data, B4_e_reconstructed_data, B4_f_reconstructed_data, B4_g_reconstructed_data, B4_h_reconstructed_data 
        return B4_List 


    @staticmethod
    def combined_dropout(target_matrix, 
                        model_type         = 'combined_dropout',
                        variable_lr        = [1e-3, 1e-4] ,
                        batch_size         = 128,
                        num_epochs         = 5,
                        noise_factor       = [0.3,0.4],
                        dropout_rate       = [0.1,0.2],
                        input_outer_layer  = 1536,
                        inner_layer        = 64 ,
                        output_outer_layer = 100, 
                         ):
        # variables: learning rate, noise_factor, dropout_rate 

        # Outputs B4_a, B4_b, B4_c, B4_d, B4_e, B4_f, B4_g, B4_h 
        print("Control Point | staticmethod combined_dropout | 4.0") 
        X0_a_manager = AutoencoderManager(  model_type   = model_type,
                                    lr           = variable_lr[0], 
                                    batch_size   = batch_size, 
                                    num_epochs   = num_epochs, 
                                    noise_factor = noise_factor[0],
                                    dropout_rate = dropout_rate[0], 
                                    outer_layer         = input_outer_layer, 
                                    inner_layer         = inner_layer,
                                    output_outer_layer  = output_outer_layer ,
                                    ) 
        print("Control Point | staticmethod combined_dropout | 4.1") 
        X0_a_manager.load_data( data_matrix = target_matrix ) 
        print("Control Point | staticmethod combined_dropout | 4.2") 
        X0_a_manager.train_combined_dropout() 
        print("Control Point | staticmethod combined_dropout | 4.3") 
        X0_a_reconstructed_data = X0_a_manager.get_reconstructed_data_combined_dropout(target_matrix) 
        print("Control Point | staticmethod combined_dropout | 4.4") 

        B4_List = X0_a_reconstructed_data 
        return B4_List 


In [10]:
class full_autoencoder():

    def __init__(self,
                    target_matrix, 
                     lr           = 1e-3, 
                     batch_size   = 128, 
                     num_epochs   = 5, 
                     noise_factor = 0.3,
                     dropout_rate = 0.2, 
                     outer_layer = 1536, 
                     inner_layer = 64): 
        self.target_matrix = target_matrix
        self.lr           = lr
        self.batch_size   = batch_size
        self.num_epochs   = num_epochs
        self.noise_factor = noise_factor
        self.dropout_rate = dropout_rate
        self.outer_layer  = outer_layer
        self.inner_layer  = inner_layer
    
    
    def full_setup(self):
        self.basic = self.setup_basic() 
        self.denoising = self.setup_denoising()
        self.dropout = self.setup_dropout()
        self.combined = self.setup_combined() 
        return self.basic, self.denoising, self.dropout, self.combined 

    def setup_basic(self): 
        # Initialize the manager for a specific autoencoder type
        manager_basic = AutoencoderManager(model_type    = 'basic', 
                                                lr           = self.lr, 
                                                batch_size   = self.batch_size, 
                                                num_epochs   = self.num_epochs, 
                                                noise_factor = self.noise_factor,
                                                dropout_rate = self.dropout_rate, 
                                                outer_layer  = self.outer_layer, 
                                                inner_layer  = self.inner_layer) 
        manager_basic.load_data( data_matrix = self.target_matrix, 
                                ) 
        manager_basic.train() 
        encoded_representations = manager_basic.get_encoded_representations(self.target_matrix)
        reconstructed_data      = manager_basic.get_reconstructed_data(self.target_matrix)
        # Return 
        return reconstructed_data 


    def setup_denoising(self): 
        # Initialize the manager for a specific autoencoder type
        manager_denoising = AutoencoderManager(model_type    = 'denoising', 
                                                lr           = self.lr, 
                                                batch_size   = self.batch_size, 
                                                num_epochs   = self.num_epochs, 
                                                noise_factor = self.noise_factor,
                                                dropout_rate = self.dropout_rate, 
                                                outer_layer  = self.outer_layer, 
                                                inner_layer  = self.inner_layer) 
        manager_denoising.load_data( data_matrix = self.target_matrix, 
                                ) 
        manager_denoising.train() 
        encoded_representations = manager_denoising.get_encoded_representations(self.target_matrix)
        reconstructed_data      = manager_denoising.get_reconstructed_data(self.target_matrix)
        # Return 
        return reconstructed_data 
    

    def setup_dropout(self):
        # Initialize the manager for a specific autoencoder type
        manager_dropout = AutoencoderManager(model_type    = 'dropout', 
                                                lr           = self.lr, 
                                                batch_size   = self.batch_size, 
                                                num_epochs   = self.num_epochs, 
                                                noise_factor = self.noise_factor,
                                                dropout_rate = self.dropout_rate, 
                                                outer_layer = self.outer_layer, 
                                                inner_layer = self.inner_layer) 
        manager_dropout.load_data( data_matrix = self.target_matrix, 
                                ) 
        manager_dropout.train() 
        encoded_representations = manager_dropout.get_encoded_representations(self.target_matrix)
        reconstructed_data      = manager_dropout.get_reconstructed_data(self.target_matrix)
        # Return 
        return reconstructed_data 
    
    def setup_combined(self):
        # Initialize the manager for a specific autoencoder type
        manager_dropout = AutoencoderManager(model_type    = 'combined', 
                                                lr           = self.lr, 
                                                batch_size   = self.batch_size, 
                                                num_epochs   = self.num_epochs, 
                                                noise_factor = self.noise_factor,
                                                dropout_rate = self.dropout_rate, 
                                                outer_layer = self.outer_layer, 
                                                inner_layer = self.inner_layer) 
        manager_dropout.load_data( data_matrix = self.target_matrix, 
                                ) 
        manager_dropout.train() 
        encoded_representations = manager_dropout.get_encoded_representations(self.target_matrix)
        reconstructed_data      = manager_dropout.get_reconstructed_data(self.target_matrix)
        # Return 
        return reconstructed_data 